In [1]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 13.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import builtins
from typing import List

# Set Up + Useful Functions

In [3]:
zero_ket = basis(2,0)
zero_rho = zero_ket * zero_ket.dag()
one_rho = basis(2,1) * basis(2,1).dag()
I = lambda: qeye(2)


def initial_rho(num_qubits):
  return tensor([zero_rho for i in range (num_qubits)])


def bell_state(state):
  """
    returns requested bell state
  """
  if state == 'phi+':
      phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_plus_ket * phi_plus_ket.dag()

  elif state == 'phi-':
      phi_minus_ket = (tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_minus_ket * phi_minus_ket.dag()

  elif state == 'psi+':
      psi_plus_ket = (tensor(basis(2, 0), basis(2, 1)) + tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_plus_ket * psi_plus_ket.dag()

  elif state == 'psi-':
      psi_minus_ket = (tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_minus_ket * psi_minus_ket.dag()

  else:
      raise ValueError("Invalid Bell state label")


def pad_op_to_left(op, num_identities):
  if num_identities > 0:
    left_op = I()
    for i in range(num_identities - 1):
      left_op = tensor(left_op, I())
    return tensor(left_op, op)
  else:
    return op


def pad_op_to_right(op, num_identities):
  if num_identities > 0:
    right_op = I()
    for i in range(num_identities - 1):
      right_op = tensor(right_op, I())
    return tensor(op, right_op)
  else:
    return op


def state_inserter(rho_initial, target_qubits, state):
  """
    Takes a density matrix of a network, inserts the state 'state' into the slots of the target_qubits.
    Assumes that target qubits are beside eachother and that no entanglement exists between
    qubits at either side of the target_qubits.

    Returns full density matrix of updated network
  """
  if isinstance(target_qubits, int):
    num_qubits_left = target_qubits
    num_qubits_right = num_qubits - target_qubits - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + 1 for i in range(num_qubits_right)]

  elif isinstance(target_qubits, builtins.list):
    num_qubits_left = target_qubits[0]
    num_qubits_right = num_qubits - target_qubits[-1] - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + len(target_qubits) for i in range(num_qubits_right)]

  else:
    raise TypeError('Invalid data type for target_qubit, must be int or list')

  rho = state
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)

  return rho


def time_for_link(eta_eff, T_p):
  """
  Gets the time taken to generate entanglement between 2 adjacent nodes
  Does this by generating random samples from a geometric distribution with
  success probability eta_eff and then multiplying by time taken for one trial.

  Parameters:
  eta_eff (float): Success probability for the geometric distribution.
  num_samples (int): Number of random samples to generate.

  Returns:
  np.ndarray: Array of random samples from the geometric distribution.
  """
  no_of_trials = np.random.geometric(eta_eff)
  time_per_trial = T_p + 2 * d / c
  time_per_link = no_of_trials * time_per_trial
  return time_per_link


def dephasing_channel(rho, t, left_padding, right_padding):
  dp_prob = (1 - np.exp(-t / T_dp)) / 2
  dp_op = np.sqrt(1 - dp_prob) * qeye(2) + np.sqrt(dp_prob) * sigmaz()
  dp_op = pad_op_to_left(dp_op, left_padding)
  dp_op = pad_op_to_right(dp_op, right_padding)
  rho_t = dp_op * rho * dp_op.dag()
  return rho_t

def dark_counts_entang_gen(rho_initial, q1, q2, n, P_link):
  """
  q2 experiences the dark counts. q1 is included in the partial trace as
  entanglement is broken otherwise
  n = total probability that a pair is established = P_link * n_ch(d)
  NOTE: This only does one target_qubit
  """
  # channel efficiency
  n_ch = lambda L: np.exp(-L/L_att)

  # total probability that a pair is established
  n = P_link * n_ch(d)

  # the chance for a detector to click (including dark counts)
  n_eff = 1 - (1 - n) * (1 - p_d) ** 2

  # given a click occurs, the probability it is from a real event
  alpha = lambda n: (n * (1 - p_d)) / n_eff

  dark_count_rho = rho_initial.ptrace([q1, q2]) * alpha(n) + (1 - alpha(n)) / 2 * tensor(rho_initial.ptrace(q1) , I())
  rho = state_inserter(rho_initial = rho_initial,
                      target_qubits = [q1,q2], state = dark_count_rho)
  return rho


def initial_F_rho(rho_initial, target_qubits: List[int], F_initial: float):
  rho_bell = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                    bell_state('psi+') +
                                                                    bell_state('psi-'))
  rho = state_inserter(rho_initial=rho_initial, target_qubits=target_qubits, state=rho_bell)
  return rho

# Entanglement Generation and Swapping

In [4]:
def entanglement_generation(rho_initial, target_qubits, F_initial, n, P_link, T_p):
  """
    This function generates entanglement across 2 neighbouring repeater stations
    q1 and q2. It does this by first creating a phi+ bell state density matrix
    across 2 qubits (with a fidelity F_initial). This is inserted into the initial
    state rho_initial by taking the partial trace of rho_initial to the left of the
    pair and to the right of the pair and then tensoring these density matrices together.

    (This function assumes that no entangled pair exists with one qubit on each
    side of the 2 selected qubits which I don't believe happens in any networking
    protocol)
  """
  # SET UP
  # getting the number of qubits either side of the pair and making a list of them
  q1, q2 = target_qubits[0], target_qubits[1]
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING PHI+ BELL PAIR WITH FIDELITY F_INITIAL
  rho = initial_F_rho(rho_initial=rho_initial, target_qubits=[q1, q2], F_initial=F_initial)

  # ADDING MEMORY NOISE
  # dephasing channel for q1 (stays in memory for twice as long)
  rho = dephasing_channel(rho=rho, t = 2 * d/c, left_padding = num_qubits_left, right_padding = num_qubits_right + 1)

  # dephasing channel for q2
  rho = dephasing_channel(rho=rho, t = d/c, left_padding = num_qubits_left + 1, right_padding = num_qubits_right)

  # ADDING DARK COUNTS NOISE
  rho = dark_counts_entang_gen(rho_initial=rho, q1=q1, q2=q2, n=n, P_link=P_link)

  return rho

In [5]:
def entanglement_swapping(rho_initial, q0q1, q2q3, lambda_BSM):
  """
  rho_initial = denisty matrix
  q0q1 = list of length 2
  q2q3 = list of length 2
  lambda_BSM = float [0,1], bell state measurement ideality
  """
  # set up
  q0, q1 = q0q1[0], q0q1[1]
  q2, q3 = q2q3[0], q2q3[1]
  num_qubits_left = q0
  num_qubits_right = (num_qubits - 1) - q3
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]
  seperation = (q3 - q0) - 1
  bell_measure_op = tensor(I(), bell_state('phi+'), I())

  # getting the initial state of the 4 qubits and creating 4 qubit density matrix
  rho_q0q1 = rho_initial.ptrace(q0q1)
  rho_q2q3 = rho_initial.ptrace(q2q3)
  rho_initial_target = tensor(rho_q0q1, rho_q2q3)

  # projecting into phi+ bell state
  rho = (bell_measure_op * rho_initial_target * bell_measure_op.dag()).ptrace([0,3]).unit()

  # adding BSM error
  rho = lambda_BSM * rho + (1 - lambda_BSM) / 4 * tensor(I(), I())

  # adding qubits to left of state
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  # adding the qubits between the newly entangled qubits q0, q3 to the right
  rho_middle = tensor([zero_rho for i in range(seperation)])
  rho = tensor(rho, rho_middle)

  # swapping q3 to the end of the tensor product so that now rho_middle is indeed in the middle
  perm_order = [i for i in range(q3+1)]
  q3_index = perm_order.pop(num_qubits_left+1) # remove the index where q3 is now from list (where q1 normally is)
  perm_order.append(q3_index) # append q3 index to end of list, moving q3 to the end
  rho = rho.permute(perm_order)

  # adding qubits to the right of state
  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)

  return rho

# Defining Global Parameters

In [6]:
# DEFINING GLOBAL PARAMETERS
# Geometric Set Up
num_qubits = 4
total_L = 100
d = total_L/(num_qubits - 1)

# Optical Fiber Properties
L_att = 22e3       # attenuation length of optical fiber
c = 2e8            # speed of light in fiber optic

# Global Error Parameters
T_dp = 100         # dephasing time constant (memory error)
p_d = 0            # probability a dark count will occur in a detection window

# DEFINING ENTANGLEMENT GENERATION PARAMETERS
entang_gen_args = {
    'n' : 1,             # probability a link is established between 2 neighbouring repeater stations
    'P_link' : 1,        # probability a link is established without considering distance based losses
    'T_p' : 0            # time to prepare an entangled pair
}

# Example Usage

In [7]:
num_qubits = 8
total_L = 100
d = total_L/(num_qubits - 1)

# simulating an 8 node network, entangling the 2 end nodes
rho = initial_rho(num_qubits)
rho = entanglement_generation(rho_initial=rho, target_qubits=[0,1], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[2,3], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[4,5], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[6,7], F_initial = 1, **entang_gen_args)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [0,1], q2q3=[2,3], lambda_BSM= 1)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [4,5], q2q3=[6,7], lambda_BSM= 1)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [0,3], q2q3=[4,7], lambda_BSM= 1)

# PRINTING OUT PARTIAL TRACE OF ALL QUBITS
print('PARTIAL TRACE OF ALL QUBITS')
for i in range(num_qubits):
  print(f'Qubit{i}\n', np.round(rho.ptrace(i).full(),3), '\n')

# PRINTING OUT 2 QUBIT DENSITY MATRIX OF ENTANGLED STATE
entangled_state = rho.ptrace([0,num_qubits-1])
print('STATE OF ENTANGLED END NODES \n',np.round(entangled_state.full(), 3))

# TESTING FIDELITY
#phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
#print('\nFidelity:', np.round(phi_plus_ket.dag() * state_03 * phi_plus_ket, 3))

PARTIAL TRACE OF ALL QUBITS
Qubit0
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Qubit1
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit2
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit3
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit4
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit5
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit6
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit7
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

STATE OF ENTANGLED END NODES 
 [[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]
